In [ ]:
from elasticsearch import Elasticsearch
from datetime import datetime
from flask import Flask, render_template, request
import csv


In [ ]:

es = Elasticsearch([{'host': 'localhost', 'port':9200, 'scheme': 'http'}], http_auth=("vijay", "password"))
es.info()

In [ ]:
cnbc_documents = list()
guardian_documents = list()
reuters_documents = list()

with open("cnbc_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        cnbc_documents.append(data)


with open("guardian_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        guardian_documents.append(data)


with open("reuters_headlines.csv", "r", encoding='utf-8') as file:

    reader = csv.reader(file)

    headers = next(reader)

    for row in reader:

        data = dict()

        i = 0

        for h in headers:

            data[h] = row[i]
            i += 1
        
        reuters_documents.append(data)


print(cnbc_documents[:10])
print()

print(guardian_documents[:10])
print()

print(reuters_documents[:10])
print()

In [ ]:
def index_documents(index, id, document):
    
    resp = es.index(index=index, id=id, document=document)


In [ ]:
for i in range(len(cnbc_documents)):
    
    es.index(index = "cnbc_index", id = i + 1, document = cnbc_documents[i])

for i in range(len(guardian_documents)):
    
    es.index(index = "guardian_index", id = i + 1, document = guardian_documents[i])

for i in range(len(reuters_documents)):
    
    es.index(index = "reuters_index", id = i + 1, document = reuters_documents[i])

In [ ]:
resp = es.get(index="cnbc_index", id=1)
print(resp['_source'])
print()

resp = es.get(index="guardian_index", id=1)
print(resp['_source'])
print()

resp = es.get(index="reuters_index", id=1)
print(resp['_source'])
print()

In [ ]:
def search_query(q):
    
    body = {
        "size": 20,
        "query": {
            "dis_max": {
                "queries": [
                    {"match": {"Headlines": q}},
                    {"match": {"Description": q}}
                ],
                "tie_breaker": 0.3
            }
        }
    }

    resp = es.search(index="cnbc_index", body=body)

    results = []

    for hit in resp['hits']['hits']:
        
        headline = hit["_source"].get('Headlines')
        time = hit["_source"].get('Time')
        description = hit["_source"].get('Description')

        formatted_result = f"Headline: {headline}\nTime: {time}\nDescription: {description}\n\n"

        print(formatted_result)
        
        formatted_result = formatted_result.replace('\n', '<br>')

        results.append(formatted_result)

    return results


In [ ]:
search_query("Jim")

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def homepage():

    results = []

    if request.method == 'POST':

        query = request.form.get('search_query')
        
        results = search(query)
    
    return render_template('test.html', results=results)


def search(query):

    return search_query(query)


app.run(debug=False)